In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#get_ipython().run_line_magic('matplotlib', 'inline')
get_ipython().run_line_magic('matplotlib', 'qt5')


import matplotlib.ticker as ticker
import os

plt.close("all")

resistencia=50   #50 ohms o 1000000 ohms 1Mohm



def procesar_csv(archivo, tiempo_col='Tiempo (ns)', voltaje_col='Voltaje (mV)', threshold_factor=-0.5):
    # Cargar datos
    datos = pd.read_csv(archivo)
    tiempo = datos[tiempo_col].values
    voltaje = datos[voltaje_col].values
    
    # Estimación de línea de base sin asumir que está al inicio
    percentil_25 = np.percentile(voltaje, 25)  # Primer cuartil
    percentil_75 = np.percentile(voltaje, 75)  # Tercer cuartil
    iqr = percentil_75 - percentil_25  # Rango intercuartil
    
    # Filtrar valores dentro de un rango razonable de la línea de base
    base_mask = (voltaje > percentil_25 - 1.5 * iqr) & (voltaje < percentil_75 + 1.5 * iqr)
    voltaje_base = voltaje[base_mask]
    
    # Calcular la media de la línea de base filtrada y centrar en 0
    media_base = np.mean(voltaje_base)
    voltaje_sin_base = voltaje - media_base

    # Graficar la señal original y la filtrada
    plt.figure(figsize=(10, 5))
    plt.plot(tiempo, voltaje, label="Señal original", alpha=0.7)
    plt.axhline(y=media_base, color='r', linestyle='--', label="Línea de base estimada")
    plt.plot(tiempo, voltaje_sin_base, label="Señal sin línea de base", color='g')
    plt.legend()
    plt.xlabel("Tiempo (ns)")
    plt.ylabel("Voltaje (mV)")
    plt.title(f"Señal procesada - {os.path.basename(archivo)}")
    plt.show()
    
    # Ajuste de umbral para detectar picos negativos
    std_base = np.std(voltaje_base)
    threshold = media_base + threshold_factor * std_base  # Umbral ajustado
    
    # Identificar el pico negativo
    indices_pico = np.where(voltaje_sin_base < threshold)[0]
    if len(indices_pico) == 0:
        print(f"No se encontró un pico claro en {archivo}")
        return None
    
    # Definir el inicio y el final del pico
    inicio_pico, fin_pico = indices_pico[0], indices_pico[-1]
    
    # Calcular carga integrando el área del pico negativo
    tiempo_pico = tiempo[inicio_pico:fin_pico+1]
    voltaje_pico = voltaje_sin_base[inicio_pico:fin_pico+1]
    carga = -simpson(voltaje_pico, tiempo_pico) / resistencia  # Negativo para considerar carga positiva
    
    return carga


def procesar_carpeta(carpeta):
    cargas = []
    archivos = [f for f in os.listdir(carpeta) if f.endswith('.csv')]
    for archivo in archivos:
        ruta = os.path.join(carpeta, archivo)
        carga = procesar_csv(ruta)
        if carga is not None:
            cargas.append(carga)
    
    # Graficar histograma de cargas
    plt.figure()
    plt.hist(cargas, bins=20, edgecolor='black', alpha=0.7)
    plt.xlabel('Carga (mV ns /50ohms)')
    plt.ylabel('Frecuencia')
    plt.title('Histograma de Cargas')
    plt.show()
    
    return cargas

# Ejemplo de uso
#os.chidr(r"C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\adquisiciones\PMT\señalesprueba")
carpeta_csv = r"C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\adquisiciones\PMT\señalesprueba"  # Cambiar por la ruta real de la carpeta
cargas = procesar_carpeta(carpeta_csv)
print(f"Cargas calculadas: {cargas}")


Cargas calculadas: [np.float64(127.9965666159645), np.float64(137.72744538503727), np.float64(121.82412563232663)]
